# **SIMS: jupyter notebook for simple training and inference**

This notebook is designed to assist in running SIMS to perform basic training, or for cell type inference using our pre-trained model checkpoints. For more advanced customization and detailed instructions on running SIMS through the API, please refer to the SIMS GitHub repository: https://github.com/braingeneers/SIMS

## Getting started

To get started, locate and run the setup_sims_env.sh file within this directory to set up your virtual environment. This ensures that SIMS runs with the correct software dependencies. After executing the script, make sure to select the newly created environment as the kernel for this notebook. It is essential to use Python 3.9.

Next, download the following libraries:

In [ ]:
from scsims import SIMS
from pytorch_lightning.loggers import WandbLogger
import anndata as an

## Training

**NOTE:** Use this section to train your own model using a labeled dataset. If you would just like to make predictions using a pretrained model checkpoint from our checkpoint folder, please skip to the Inference section.

*For custom training jobs and logging, we recommend running SIMS via an API. Please refer to our GitHub for instructions: https://github.com/braingeneers/SIMS*

#### Load training data, initialize SIMS model, and train. 

In [ ]:
labeled_data_path = "/my_labeled_data.h5ad"  # Replace 'my_labeled_data.h5ad' with the 
                                             # data you would like to use for training
                                            
train_data = an.read_h5ad(labeled_data_path)

sims = SIMS(data=train_data, class_label='class_label')  # Change 'class_label' to be the variable you are predicting for

sims.train()  # Begin training the model. Checkpoints will save automatically  
              # to a 'lightning_logs' directory in the current working directory

## Inference

#### Load data and initialize SIMS model

In [ ]:
checkpoint_path = '../checkpoint/myawesomemodel.ckpt'   # Replace '../checkpoint/myawesomemodel.ckpt' with the path to a
                                                        # pretrained model from the sims_app/checkpoint folder. Alternatively, 
                                                        # you can load your own .ckpt file developed during training from the 
                                                        # 'lightning_logs' directory  
                                                        
test_data_path = '/my_test_data.h5ad'  # Replace 'my_test_data.h5ad' with the data you would like
                                       # to perform predictions on using your chosen model checkpoint

sims = SIMS(weights_path=checkpoint_path)  # Initialize the SIMS model with the checkpoint

#### Run and save predictions

In [ ]:
cell_predictions = sims.predict(test_data_path)  # Run predictions

cell_predictions.to_csv('predictions.csv')  # Save predictions

#### View the explainability of the model

In [ ]:
explainability_matrix = sims.explain(test_data_path)  # Generate explainability matrix